In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [ ]:
import os


root_dir = os.path.abspath('../bailian_nlp/')

data_path = os.path.join(root_dir, 'datadrive/bailian/pos')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'train.csv')
valid_path = os.path.join(data_path, 'valid.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/chinese_L-12_H-768_A-12/')
init_checkpoint_pt = os.path.join(model_dir, 'bert_model.bin')
bert_config_file = os.path.join(model_dir, 'bert_config.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')
config_file = os.path.join(model_dir, 'pos.json')



In [ ]:
from bailian_nlp.modules.data.tokenization import BailianTokenizer

tokenizer = BailianTokenizer(vocab_file)

text = '这是个好人。are you ok Málaga'

tokenizer.tokenize(text)

In [ ]:
unicodedata.normalize('NFD', text)

In [ ]:
from bailian_nlp.modules.data.tokenization import BailianTokenizer
tokenizer = BailianTokenizer(vocab_file)
text = 'áf 3 are you '
text = 'http://www.xteacher.com'
text = '中华人'
text = '今年在韩国举办的「二十一世纪的计算」学术研讨会以人机协作（HumanandMachineWorkingasaTeam）为主题，现任微软全球资深副总裁，微软亚太研发集团主席，兼微软亚洲研究院院长洪小文博士做了题目为《人工智能与人类智能的共同进化》（Co-EvolutionofArtificialIntelligenceandHumanIntelligence）的演讲。'
tokens, marker = tokenizer.tokenize(text)
# 切词
print(len(text), tokens, marker)
# 还原
for i,(_, (st, ed)) in enumerate(marker):
    print(i, text[st:ed])


In [ ]:
tokenizer.recover_text(text, tokens, ['x', 's', '3'], marker)

In [ ]:
raw_data_dir = os.path.dirname(data_path)
seg_file = os.path.join(raw_data_dir, 'final_baidu-23w.txt')
seg_f = open(seg_file)

pos_sents = seg_f.readlines()
pos_sents[0]

In [ ]:
from pytorch_pretrained_bert import tokenization
tokenization._is_punctuation('\n')

In [ ]:
idx = 0
counter = 0
while idx < len(pos_sents):
    try:
        idx += 1
        pos_sent = pos_sents[idx]
        tokenizer = BailianTokenizer(vocab_file)
        tokenizer.tokenize_with_pos_text(pos_sent)
    except:
        counter += 1
        print(counter, idx, counter/(idx+1))
#         raise

counter

In [ ]:
idx, counter

In [ ]:
pos_sent

In [ ]:
tokenizer.tokenize_with_pos_text(pos_sents[idx-1])

In [ ]:
def build_data():
    
    import re
    
    p = re.compile(r'(.+?)/(?:([a-z]{1,2})(?:$| ))')
    
    raw_data_dir = os.path.dirname(data_path)
    seg_file = os.path.join(raw_data_dir, 'final_baidu-23w.txt')
    fake_file = os.path.join(raw_data_dir, 'fake.txt')
    special_file = os.path.join(raw_data_dir, 'special.txt')
    dict_file = os.path.join(raw_data_dir, 'single.txt')
    
    delimiter='△△△'
    
    replace_chars = [
        '\x97',
        '\uf076',
        "\ue405",
        "\ue105",
        "\ue415",
        '\x07',
        '\x7f',
        '\u3000',
        '\xa0',
        ' '
    ]
    with open(seg_file) as fin1, \
          open(fake_file) as fin2, \
          open(special_file) as fin3, \
          open(dict_file) as fin4, \
          open(train_path, 'w') as train_f, \
          open(valid_path, 'w') as valid_f:
        
        train_f.write(f'0{delimiter}1\n')
        valid_f.write(f'0{delimiter}1\n')
        
        fins = [fin1, fin2, fin3, fin4]
        for k, fin in enumerate(fins):
            for line in fin:
                line = line.strip()
                if not line:
                    continue

                import random
                score = random.random()

                if k < 2:
                    fout = train_f if score > 0.006 else valid_f
                else:
                    fout = train_f
                
                words = []
                flags = []
                for word, flag in p.findall(line):
                    from bailian_nlp.modules.data.tokenization import _is_control
                    
                    char_list = [
                        c for c in list(word)
                        if not (c in replace_chars or c.isspace() or _is_control(c))
                    ]
                    char_size = len(char_list)
                    if char_size == 1:
                        # 一些错误的单个字符实体剔除掉
                        if flag in ['nt', 'ti', 'nr', 'ns', 'nz']:
                            flag = 'xx'
                        # 单个
                        tag_list = [f'S_{flag}']
                    elif char_size == 0:
                        continue
                    else:
                        tag_list = [f'B_{flag}'] + [f'I_{flag}']  * (len(char_list) - 2) + [f'E_{flag}']

                    if char_size != len(tag_list) or char_size == 0:
                        print(line)
                        print(word, flag)
                        print(char_list, tag_list)
                        

                    words.extend(char_list)
                    flags.extend(tag_list)

                assert len(words) == len(flags)

                fout.write(delimiter.join([
                    ' '.join(flags),
                    ' '.join(words)
                ]))
                fout.write('\n')
            
            
            
build_data()
    

In [ ]:
# 正常训练

from bailian_nlp.modules import BertNerData as NerData

data = NerData.create(
    train_path,
    valid_path, 
    vocab_file,
    data_type="bert_uncased",
    is_cls=False,
    max_seq_len=128,
    batch_size=128
    
)

import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    bert_config_file, 
    init_checkpoint_pt,
    enc_hidden_dim=256
)
model.get_n_trainable_params()


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 10
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')


In [ ]:
num_epochs = 10
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')

In [ ]:
# 恢复训练

from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()
tagger.init_env(for_train=True)

data = tagger.learner.data
learner = tagger.learner
num_epochs = 1
learner.load_model()
learner.t_total = num_epochs * len(data.train_dl)
learner.sup_labels = list(set(data.id2label[1:]) | set(learner.sup_labels))
learner.fit(num_epochs, target_metric='f1')


In [ ]:
learner.fit(num_epochs, target_metric='f1')

In [ ]:
from modules.data import bert_data
reload(bert_data)
dl = bert_data.get_bert_data_loader_for_predict(valid_path, learner)

In [ ]:
# learner.load_model()
preds = learner.predict(dl)

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds)
print(clf_report)

In [ ]:
from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()


In [ ]:
import time

st = time.time()
text = '近日，编程猫（深圳点猫科技有限公司）正式对外宣布完成B轮1.2亿元融资。本轮融资由高瓴资本领投，清流资本、清晗基金跟投，天使轮投资者猎豹移动继续跟投。'
# text = '未来编程教育产业将蓬勃发展，编程猫作为提供工具与内容的企业，有望长期处于行业领跑者地位。'
# text = '美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司始创于2004年,是中国健康体检和医疗服务集团,总部位于上海,深耕布局北京、深圳、沈阳、广州、成都、武汉、...'
# text = '百炼智能百炼智能'
# text = '高越君冯是聪'
text = '周光明确否认CEO佟显侨和衡量推动发出公司公告'
text = '周光明确否认CEO佟显侨和CTO衡量说的罪名'
text = '董事'
# text = '一言九鼎'
# text = '客户包括雀巢、洲际酒店、瑞士航空、德意志银行、红牛、瑞士联合银行等世界知名公司。'
# text = '药方越是多的，越表明病是难的于治疗'
text = '2012-11-8 的客户包括：宝洁 、杜邦、强生、百事可 乐、3M 、三菱、吉列、通用医疗系统、美国 运通、雅芳、纽约银 行、百时美—施贵 宝、礼来、迪斯尼、纳贝斯克、纽约'
res = tagger.cut(text)
ed = time.time()
print(ed - st)
res